In [1]:
!pip install qrcode[pil]

  Using cached qrcode-8.2-py3-none-any.whl (45 kB)


You should consider upgrading via the 'C:\github\personal\lyrics-Serenate\util\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
#!/usr/bin/env python3
import argparse
import sys
from typing import Optional
import qrcode
from qrcode.constants import ERROR_CORRECT_L, ERROR_CORRECT_M, ERROR_CORRECT_Q, ERROR_CORRECT_H
from PIL import Image, ImageOps, ImageDraw

EC_MAP = {
    "L": ERROR_CORRECT_L,
    "M": ERROR_CORRECT_M,
    "Q": ERROR_CORRECT_Q,
    "H": ERROR_CORRECT_H,
}



In [6]:
def add_logo_to_qr(qr_img: Image.Image,
                   logo_path: str,
                   logo_scale: float = 0.2,
                   logo_border: int = 8,
                   logo_round: bool = True,
                   border_color: str = "white") -> Image.Image:
    """
    Inserta un logo centrado dentro del QR.
    - logo_scale: proporción del lado del QR (0.1–0.3 recomendado).
    - logo_border: grosor del borde blanco que mejora la lectura del QR.
    - logo_round: si True, redondea el logo (círculo o esquinas redondas).
    """
    qr = qr_img.convert("RGBA")
    logo = Image.open(logo_path).convert("RGBA")

    # calcular tamaño del logo relativo al QR
    qrw, qrh = qr.size
    max_logo_w = int(qrw * logo_scale)
    max_logo_h = int(qrh * logo_scale)

    # preservar aspecto
    logo.thumbnail((max_logo_w, max_logo_h), Image.LANCZOS)
    lw, lh = logo.size

    # borde blanco detrás del logo para mejorar contraste
    # creamos un "lienzo" un poco más grande que el logo
    canvas_w = lw + 2 * logo_border
    canvas_h = lh + 2 * logo_border
    canvas = Image.new("RGBA", (canvas_w, canvas_h), (255, 255, 255, 0))

    # máscara para forma redondeada si se pide
    if logo_round:
        # círculo si es casi cuadrado; si no, esquinas redondas
        radius = min(canvas_w, canvas_h) // 2
        mask = Image.new("L", (canvas_w, canvas_h), 0)
        draw = ImageDraw.Draw(mask)
        if abs(lw - lh) <= max(2, int(0.1 * max(lw, lh))):
            # círculo
            draw.ellipse((0, 0, canvas_w, canvas_h), fill=255)
            # fondo blanco circular
            bg = Image.new("RGBA", (canvas_w, canvas_h), border_color)
            canvas = Image.composite(bg, canvas, mask)
            # pegar logo centrado usando máscara circular recortada al tamaño del logo
            # (logo centrado en el círculo)
            logo_mask = Image.new("L", logo.size, 255)
            canvas.paste(logo, (logo_border, logo_border), logo_mask)
        else:
            # esquinas redondeadas
            corner = int(min(canvas_w, canvas_h) * 0.25)
            draw.rounded_rectangle((0, 0, canvas_w, canvas_h), radius=corner, fill=255)
            bg = Image.new("RGBA", (canvas_w, canvas_h), border_color)
            canvas = Image.composite(bg, canvas, mask)
            canvas.paste(logo, (logo_border, logo_border), logo)
    else:
        # borde rectangular
        bg = Image.new("RGBA", (canvas_w, canvas_h), border_color)
        canvas.alpha_composite(bg)
        canvas.paste(logo, (logo_border, logo_border), logo)

    # pegar en el centro del QR
    pos = ((qrw - canvas_w) // 2, (qrh - canvas_h) // 2)
    qr.alpha_composite(canvas, dest=pos)
    return qr.convert("RGB")

In [7]:
def make_qr(text: str,
            out_path: str,
            ec: str = "H",
            box_size: int = 10,
            border: int = 4,
            version: Optional[int] = None,
            logo_path: Optional[str] = None,
            logo_scale: float = 0.2,
            logo_border: int = 8,
            logo_round: bool = True,
            border_color: str = "white") -> str:
    error_correction = EC_MAP.get(ec.upper(), ERROR_CORRECT_H)

    qr = qrcode.QRCode(
        version=version,
        error_correction=error_correction,
        box_size=box_size,
        border=border,
    )
    qr.add_data(text)
    qr.make(fit=True)

    img = qr.make_image(fill_color="black", back_color="white").convert("RGB")

    if logo_path:
        img = add_logo_to_qr(
            img,
            logo_path=logo_path,
            logo_scale=logo_scale,
            logo_border=logo_border,
            logo_round=logo_round,
            border_color=border_color,
        )

    img.save(out_path)
    return out_path

In [11]:
img = make_qr(
    text="https://diego-cb.github.io/lyrics-Serenate/",
    out_path="./qr.png",
    ec="H",
    logo_path="../src/assets/smporres.jpeg",
    logo_scale=0.2,           # 0.15–0.25 works well
    logo_border=10,           # white border around the logo (px)
    logo_round=True,          # rounded logo container
)
